In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import psutil
import geopandas as gpd
import shapely
from shapely.geometry import Point
from collections import Counter
import folium
import requests
from bs4 import BeautifulSoup as BS

In [8]:
sf2009 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2009singlefamily_final_with_sqft.txt')
sf2013 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2013singlefamily_final_with_sqft.txt')
sf2017 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2017singlefamily_final_with_sqft.txt')
parcels = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/zoneparcel.xlsx')
tax = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2016Tax_TN.xls', skiprows=3, skipfooter=17)

In [82]:
issued = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/Issued Building Permits 2010 to Present.xlsx')

In [64]:
issued.columns

Index(['Permit #', 'Permit Type Description', 'Permit Subtype Description',
       'Parcel', 'Date Entered', 'Date Issued', 'Const. Cost', 'Address',
       'City', 'State', 'Zip', 'Subdivision/Lot', 'Contact', 'Permit Type',
       'Permit Subtype', 'IVR Trk#', 'Purpose', 'Council Dist', 'Census Tract',
       'Mapped Location'],
      dtype='object')

In [67]:
issued.shape

(36221, 20)

In [16]:
tax.drop(tax.loc[tax['ZIP\ncode [1]'] == 99999].index, inplace=True)
tax.drop(tax.loc[tax['ZIP\ncode [1]'] == 0.0].index, inplace=True)
tax.drop(tax.loc[tax['ZIP\ncode [1]'] == np.NaN].index, inplace=True)

tax = tax.dropna()


In [19]:
parcels09 = pd.DataFrame(parcels.loc[parcels['APN'].isin(sf2009.APN)])
parcels13 = pd.DataFrame(parcels.loc[parcels['APN'].isin(sf2013.APN)])
parcels17 = pd.DataFrame(parcels.loc[parcels['APN'].isin(sf2017.APN)])

In [ ]:
parcels13.info()

In [ ]:
parcels17.info()

In [21]:
rebuilt_parcels = parcels.drop(parcels.loc[parcels.DateEffective < '2000'].index, inplace=True)
rebuilt_parcels = parcels.drop(parcels.loc[parcels.IsActive == 'N'].index, inplace=True)
#rebuilt_parcels = pd.DataFrame(parcels.loc)

In [22]:
rebuilt_parcelsdtindex = rebuilt_parcels.set_index('DateEffective')

AttributeError: 'NoneType' object has no attribute 'set_index'

In [ ]:
rebuilt_parcelsdtindex.head()

In [23]:
rebuilt_parcelsdtindex.IsActive.resample('Y').count().plot()

NameError: name 'rebuilt_parcelsdtindex' is not defined

In [ ]:
homes.columns

In [ ]:
len(homes.Zip.unique())

In [ ]:
rebuilt_parcels.DateEffective.describe()

In [ ]:
rebuilt_parcels.info()

In [ ]:
rebuilt_parcels.IsActive.unique()

In [ ]:
parcels.head()

In [ ]:
parcels.IsActive.unique()

In [ ]:
issued.Parcel.describe()

In [ ]:
rebuilt_parcels.info()

In [ ]:
sf2009.Yearly_Housing_Spending.max()

In [ ]:
sf2009.columns

In [ ]:
zipgroup09 = sf2009.groupby('Zip')

In [ ]:
zipgroup09.Yearly_Housing_Spending.mean()

In [ ]:
bucketgrouped09 = sf2009.groupby('bucket')

In [ ]:
bucketgrouped09.Yearly_Housing_Spending.max()

In [ ]:
districtgrouped09 = sf2009.groupby('DistrictNo')

In [ ]:
districtgrouped09.Yearly_Housing_Spending.describe()

In [ ]:
districtgrouped09.Yearly_Housing_Spending.mean()

In [69]:
sf2009.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2009 LAND':'LandValue', '2009 IMPR':'ImprovementValue', '2009 TOTAL APPR':'AppraisedValue', '2009 TOTAL ASSD':'2009 TOTAL ASSD'}, inplace=True)
sf2013.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2013 LAND':'LandValue', '2013 IMPR':'ImprovementValue', '2013 TOTAL APPR':'AppraisedValue', '2013 TOTAL ASSD':'2013 TOTAL ASSD'}, inplace=True)
sf2017.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2017 LAND':'LandValue', '2017 IMPR':'ImprovementValue', '2017 TOTAL APPR':'AppraisedValue', '2017 TOTAL ASSD':'2017 TOTAL ASSD'}, inplace=True)

In [88]:
issued.rename(columns={'Permit #': 'PermitNo', 'Permit Type Description':'PermTypeDesc', 'Permit Subtype Description': 'PermSubDesc', 'Parcel':'Parcel', 'Date Entered':'EnteredDt', 'Date Issued': 'IssuedDt', 'Const.Cost': 'ConstCost', 'Address':'Address', 'City':'City', 'State':'State', 'Zip':'Zip', 'Subdivision/Lot': 'Subdiv_Lot', 'Contact':'Contact', 'Per Ty': 'PermType', 'Per SubType': 'PermSubtype', 'IVR Trk#': 'IVRTrk#', 'Purpose':'Purpose', 'Council Dist':'CouncilDist', 'Census Tract':'CensusTract', 'Mapped Location':'MappedLoc'}, inplace=True)

In [89]:
issued.head()

,PermitNo,PermTypeDesc,PermSubDesc,Parcel,EnteredDt,IssuedDt,ConstCost,Address,City,State,ZIP,Subdiv_Lot,Contact,PermType,PermSubtype,IVRTrk#,Purpose,CouncilDist,CensusTract
0,200622671,Building Demolition Permit,"Retail, Department / Retail Stores",8212027000,2006-11-13,2011-03-07,7500.0,931 MAIN ST,NASHVILLE,TN,37206,PT. LOTS 4 & 5 OF D D PHILLIPS SILVERDENE PARK...,"Bubis, Harriet K., Tr.",CADM,CAF20M001,1502247,Demo non res bldg. Not to be burned or buried...,6.0,37011900.0
1,200624037,Building Commercial - New,"Religious Institution, Worship Space",7211050900,2006-12-21,2010-08-17,3553025.0,1515 ANN ST,NASHVILLE,TN,37216,LOT 1 FIRST BAPTIST SOUTH INGLEWOOD CHURCH 1ST...,Southland Constructors Llc,CACN,CAB10A305,1510889,TO CONSTRUCT A NEW SANCTUARY TO EXISTING CHUR...,7.0,37011400.0
2,200703503,Building Residential - New,Single Family Residence,12700029100,2007-01-22,2014-07-25,285372.0,8290 CHARLOTTE PIKE,NASHVILLE,TN,37221,N/S CHARLOTTE PK E OF HUNTWICK TL,"Christian, Lela",CARN,CAA01R301,1517067,to construct a 3500 sf single family residence...,35.0,37018301.0
3,200723886,Building Commercial - New,Master Permit Application,10501049100,2007-05-21,2010-09-27,1450985.0,1006 EDGEHILL AVE,NASHVILLE,TN,37203,"LOTS 24,26,40 & P/O LOT 41 CASTLE ADDN & ACREA...",Clayco Inc,CACN,CAZ03A001,1557083,Master development application for the develop...,17.0,37016200.0
4,200800263,Building Residential - New,Single Family Residence,11200014000,2008-01-04,2012-03-15,266549.0,4664 HEATH RD,NASHVILLE,TN,37221,S OF MANNING HOLLOW RD W OF OLD CHARLOTTE PK,"White, Nick Contracting, Inc",CARN,CAA01R301,1610214,TO CONSTRUCT A 2544 SQ FT SINGLE FAMILY RESIDE...,35.0,37018301.0


In [84]:
issued['EnteredDt'] = pd.to_datetime(issued['EnteredDt'])
issued['IssuedDt'] = pd.to_datetime(issued['IssuedDt'])

In [ ]:
issued.head()

In [28]:
nulls09 = sf2009.loc[sf2009['FinishedArea'] == 0].index
nulls13 = sf2013.loc[sf2013['FinishedArea'] == 0].index
nulls17 = sf2017.loc[sf2017['FinishedArea'] == 0].index

sf2009 = sf2009.drop(index=nulls09)
sf2013 = sf2013.drop(index=nulls13)
sf2017 = sf2017.drop(index=nulls17)

In [ ]:
sf2009

In [ ]:
issued.loc[issued.MappedLoc.str.match(r'\(') == True]

In [ ]:
issued.MappedLoc.searchsorted()

In [29]:
r=0.0475/12 # monthly interest rate
m=30*12 # number of months in 30 years

# calculate yearly spending

sf2009['Yearly_Housing_Spending']=12*(sf2009.AppraisedValue-sf2009.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))
sf2013['Yearly_Housing_Spending']=12*(sf2013.AppraisedValue-sf2013.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))
sf2017['Yearly_Housing_Spending']=12*(sf2017.AppraisedValue-sf2017.AppraisedValue*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))

In [ ]:
issued.columns

In [30]:
sf2009['PricePerSqFt'] = sf2009.AppraisedValue/sf2009.FinishedArea
sf2013['PricePerSqFt'] = sf2013.AppraisedValue/sf2013.FinishedArea
sf2017['PricePerSqFt'] = sf2017.AppraisedValue/sf2017.FinishedArea

In [ ]:
sf2009.info()

In [ ]:
sf2009.head()

In [ ]:
sf2009.info()

In [ ]:
sf2009.loc[sf2009.Yearly_Housing_Spending == sf2009.Yearly_Housing_Spending.max()]

In [31]:
print('30th percentile annual housing cost, 2009: ' + str(sf2009['Yearly_Housing_Spending'].quantile(0.3)))
print('30th percentile annual housing cost, 2013: ' + str(sf2013['Yearly_Housing_Spending'].quantile(0.3)))
print('30th percentile annual housing cost, 2017: ' + str(sf2017['Yearly_Housing_Spending'].quantile(0.3)))

30th percentile annual housing cost, 2009: 5191.794828125001
30th percentile annual housing cost, 2013: 5024.171434895834
30th percentile annual housing cost, 2017: 7280.291700520835


# Creating buckets for affordability calculation

In [ ]:
income_by_zip_2009=pd.read_excel('09zp43tn.xls',skiprows=3)

In [32]:
AMI_09=64900/12
AMI_13=62300/12
AMI_17=68000/12


income_cat_1_17=0.30*AMI_17
income_cat_2_17=0.60*AMI_17
income_cat_3_17=0.80*AMI_17
income_cat_4_17=1.20*AMI_17

income_cat_1_13=0.30*AMI_13
income_cat_2_13=0.60*AMI_13
income_cat_3_13=0.80*AMI_13
income_cat_4_13=1.20*AMI_13

income_cat_1_09=0.30*AMI_09
income_cat_2_09=0.60*AMI_09
income_cat_3_09=0.80*AMI_09
income_cat_4_09=1.20*AMI_09

In [34]:
cat1_spend=income_cat_1_17*0.3
cat2_spend=income_cat_2_17*0.3
cat3_spend=income_cat_3_17*0.3
cat4_spend=income_cat_4_17*0.3

print(cat1_spend, cat2_spend, cat3_spend, cat4_spend)

510.0 1020.0 1360.0000000000002 2040.0


In [35]:
def bucket(x):
    if x<=cat1_spend:
        return '(1) <30%_AMI'
    elif x<=cat2_spend:
         return '(2) 30-60%_AMI'
    elif x<=cat3_spend:
        return '(3) 60-80%_AMI'
    elif x<=cat4_spend:
        return '(4) 80-120%_AMI'
    else:
        return '(5) >120%_AMI'

In [36]:

# assign the buckets to sf dataframes

sf2013['bucket']=sf2013.Yearly_Housing_Spending.apply(bucket)

sf2009['bucket']=sf2009.Yearly_Housing_Spending.apply(bucket)

sf2017['bucket']=sf2017.Yearly_Housing_Spending.apply(bucket)

In [ ]:
sf2009.head()

In [ ]:
count13 = Counter(sf2013.bucket)
supply13 = dict(count13)

In [ ]:
count13

In [ ]:
count17 = Counter(sf2017.bucket)
supply17 = dict(count17)

In [ ]:
sf2009.loc[sf2009.Yearly_Housing_Spending == sf2009.Yearly_Housing_Spending.max()]

In [ ]:
sf2009.FinishedArea.describe()

In [ ]:
sf2009.loc[sf2009.PricePerSqFt == sf2009.PricePerSqFt.max()]

In [ ]:
sf2009.FinishedArea.plot(kind='hist')

In [ ]:
sf2009['Yearly_Housing_Spending'].describe()

In [ ]:
sf2013['Yearly_Housing_Spending'].describe()

In [ ]:
sf2017['Yearly_Housing_Spending'].describe()

In [ ]:
sf2009.Yearly_Housing_Spending.plot(kind='hist', bins=150)

In [ ]:
sf2013.info()

In [ ]:
sf2017.info()

In [ ]:
sf2009.AppraisedValue.describe()

In [ ]:
sf2009.AppraisedValue.plot()

In [ ]:
sf2013.AppraisedValue.describe()

In [ ]:
sf2013.AppraisedValue.plot()

In [ ]:
sf2017.AppraisedValue.describe()

In [ ]:
sf2017.AppraisedValue.plot()

In [ ]:
sf2009.AppraisedValue.describe()

In [ ]:
issued.head()

In [62]:
issued.IssuedDt.min()

Timestamp('2015-09-03 00:00:00')

In [ ]:
IssuedDt(demo) > IssuedDt(home) in issued where issued.PermType == CADM and issued.Address ==demos.Address

In [ ]:
dtdemissued = issued.loc[issued.PermType == 'CADM'].IssuedDt

In [100]:
demos = issued.loc[issued.PermType == 'CADM']

In [101]:
issued.loc[issued.Address.isin(demos.Address)]

,PermitNo,PermTypeDesc,PermSubDesc,Parcel,EnteredDt,IssuedDt,ConstCost,Address,City,State,ZIP,Subdiv_Lot,Contact,PermType,PermSubtype,IVRTrk#,Purpose,CouncilDist,CensusTract
0,200622671,Building Demolition Permit,"Retail, Department / Retail Stores",8212027000,2006-11-13,2011-03-07,7500.0,931 MAIN ST,NASHVILLE,TN,37206,PT. LOTS 4 & 5 OF D D PHILLIPS SILVERDENE PARK...,"Bubis, Harriet K., Tr.",CADM,CAF20M001,1502247,Demo non res bldg. Not to be burned or buried...,6.0,37011900.0
3,200723886,Building Commercial - New,Master Permit Application,10501049100,2007-05-21,2010-09-27,1450985.0,1006 EDGEHILL AVE,NASHVILLE,TN,37203,"LOTS 24,26,40 & P/O LOT 41 CASTLE ADDN & ACREA...",Clayco Inc,CACN,CAZ03A001,1557083,Master development application for the develop...,17.0,37016200.0
41,200826560,Building Use & Occupancy,Master Permit Application,8213019900,2008-07-22,2010-09-24,0.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CAUO,CAZ03A001,1649702,To authorize construction of a new 242 unit ...,19.0,37019400.0
42,200826567,Building Commercial - New,Automobile Parking - Structure,8213019900,2008-07-22,2010-09-27,7771288.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CACN,CAF03S002,1649709,To construct a new 332 space four and one-half...,19.0,37019400.0
43,200826568,Building Commercial - Shell,"Retail, Department / Retail Stores",8213019900,2008-07-22,2010-09-27,231863.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CACH,CAF20M001,1649710,"To prepare a 2,674 square foot portion of buil...",19.0,37019400.0
44,200826570,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,11065521.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CARN,CAA03R399,1649712,To construct a new 122 unit apartment building...,19.0,37019400.0
45,200826571,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,12852707.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CARN,CAA03R399,1649713,To construct a new 120 unit apartment building...,19.0,37019400.0
52,200837643,Building Commercial - Addition,"Religious Institution, Worship Space",6016009100,2008-10-27,2010-03-31,180744.0,131 DELLWAY DR,NASHVILLE,TN,37207,LOTS 272-273-274-PT 271 SEC 2 JOYWOOD HEIGHTS,"Chatman Development Group, Gordan Chatman Dba",CACA,CAB10A305,1666304,To construct a new 1360 square foot addition t...,8.0,37011001.0
57,200901927,Building Residential - Addition,Single Family Residence,9111014800,2009-01-28,2011-01-21,24000.0,5200 INDIANA AVE,NASHVILLE,TN,37209,LOTS 1101 & 1103 BLK 160 WEST NASH 1,"Woodall, Terry W.",CARA,CAA01R301,1678486,TO ADD A SECOND FLOOR TO EXISTNG NON CONFORMIN...,20.0,37013300.0
76,200904737,Building Commercial - New,"Vehicular Sales And Service Limited, S-1",9113035501,2009-03-02,2010-04-20,460297.0,5907 CHARLOTTE PIKE,NASHVILLE,TN,37209,PT LOT 4 J M BRANHAMS FARM,"Mohammedkah, Taghi Gen Contr",CACN,CAF22S001,1682957,to build a new 4500 office building on prope...,20.0,37018101.0


In [ ]:
issued.loc[issued.isin(parcels)]

In [ ]:
issued.MappedLoc.unique()

In [ ]:
issued.columns

In [ ]:
issued.PermTypeDesc.unique()

In [ ]:
issued.head()

In [110]:
stypes = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R398', 'CAA03R399']

In [ ]:
issued.loc[issued.Per]

In [145]:
residences = pd.DataFrame(issued.loc[issued.PermType == 'CARN'])
homes = pd.DataFrame(residences.loc[residences.PermSubtype.isin(stypes)])

In [161]:
homes['CostToBuyer'] = homes.ConstCost * 0.09 + homes.ConstCost

In [162]:
r=0.0475/12 # monthly interest rate
m=30*12 # number of months in 30 years

homes['Yearly_Housing_Spending']=12*(homes.CostToBuyer-homes.CostToBuyer*0.05)*((r*(1+r)**m)/((1+r)**(m-1)))

In [163]:
homes['bucket']=homes.Yearly_Housing_Spending.apply(bucket)

In [146]:
homes.head()

,PermitNo,PermTypeDesc,PermSubDesc,Parcel,EnteredDt,IssuedDt,ConstCost,Address,City,State,ZIP,Subdiv_Lot,Contact,PermType,PermSubtype,IVRTrk#,Purpose,CouncilDist,CensusTract
2,200703503,Building Residential - New,Single Family Residence,12700029100,2007-01-22,2014-07-25,285372.0,8290 CHARLOTTE PIKE,NASHVILLE,TN,37221,N/S CHARLOTTE PK E OF HUNTWICK TL,"Christian, Lela",CARN,CAA01R301,1517067,to construct a 3500 sf single family residence...,35.0,37018301.0
4,200800263,Building Residential - New,Single Family Residence,11200014000,2008-01-04,2012-03-15,266549.0,4664 HEATH RD,NASHVILLE,TN,37221,S OF MANNING HOLLOW RD W OF OLD CHARLOTTE PK,"White, Nick Contracting, Inc",CARN,CAA01R301,1610214,TO CONSTRUCT A 2544 SQ FT SINGLE FAMILY RESIDE...,35.0,37018301.0
44,200826570,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,11065521.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CARN,CAA03R399,1649712,To construct a new 122 unit apartment building...,19.0,37019400.0
45,200826571,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,12852707.0,515 MADISON ST,NASHVILLE,TN,37208,LOT 1 BRISTOL GERMANTOWN - JEFFERSON STREET PR...,Gaius Overton,CARN,CAA03R399,1649713,To construct a new 120 unit apartment building...,19.0,37019400.0
72,200902550,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",9311024900,2009-02-04,2010-02-22,13181361.0,8 ACADEMY PL,NASHVILLE,TN,37210,LOT 3 ROLLING MILL HILL PHASE 1 1ST REV,Bomar Construction Co Inc,CARN,CAA03R399,1679403,to construct a four floor 109 units total with...,19.0,37019500.0


In [130]:
homes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17862 entries, 2 to 99823
Data columns (total 22 columns):
PermitNo                   17862 non-null object
PermTypeDesc               17862 non-null object
PermSubDesc                17862 non-null object
Parcel                     17862 non-null object
EnteredDt                  17862 non-null datetime64[ns]
IssuedDt                   17862 non-null datetime64[ns]
ConstCost                  17862 non-null float64
Address                    17862 non-null object
City                       17819 non-null object
State                      17821 non-null object
ZIP                        17862 non-null int64
Subdiv_Lot                 17862 non-null object
Contact                    17862 non-null object
PermType                   17862 non-null object
PermSubtype                17862 non-null object
IVRTrk#                    17862 non-null int64
Purpose                    17862 non-null object
CouncilDist                17858 non-null f

Certain permits have construction cost of 0.  These are anomalies (amended permits, master permits, etc.) that may be ignored.

In [147]:
badpermits = homes.loc[homes.ConstCost == 0].index

In [148]:
homes.drop(badpermits, inplace = True)

In [164]:
rebuilds = pd.DataFrame(homes.loc[homes.Address.isin(demos.Address)])

In [170]:
rebuilds.head()

,PermitNo,PermTypeDesc,PermSubDesc,Parcel,EnteredDt,IssuedDt,ConstCost,Address,City,State,...,Contact,PermType,PermSubtype,IVRTrk#,Purpose,CouncilDist,CensusTract,CostToBuyer,Yearly_Housing_Spending,bucket
44,200826570,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,11065521.0,515 MADISON ST,NASHVILLE,TN,...,Gaius Overton,CARN,CAA03R399,1649712,To construct a new 122 unit apartment building...,19.0,37019400.0,12061417.89,546425.890237,(5) >120%_AMI
45,200826571,Building Residential - New,"Multifamily, Apt / Twnhome > 5 Unit Bldg",8213019900,2008-07-22,2010-09-27,12852707.0,515 MADISON ST,NASHVILLE,TN,...,Gaius Overton,CARN,CAA03R399,1649713,To construct a new 120 unit apartment building...,19.0,37019400.0,14009450.63,634678.824832,(5) >120%_AMI
484,201000299,Building Residential - New,Single Family Residence,11801012000,2010-01-07,2010-01-13,271124.0,1102 MONTROSE AVE,NASHVILLE,TN,...,Rigid Development Inc,CARN,CAA01R301,1725022,TO CONSTRUCT A NEW 2 STORY 2800 SQ FT SINGLE F...,17.0,37017000.0,295525.16,13388.359488,(5) >120%_AMI
952,201002295,Building Residential - New,Single Family Residence,11701006700,2010-02-04,2010-02-10,502719.0,611 LYNNBROOK RD,NASHVILLE,TN,...,"Castle Contractor's, Looney, Alan Dba",CARN,CAA01R301,1728126,TO CONSTRUCT A 4773 SQ FT 2 STORY SINGLE FAMIL...,24.0,37018000.0,547963.71,24824.739577,(5) >120%_AMI
972,201002415,Building Residential - New,Single Family Residence,13005010500,2010-02-05,2010-03-30,455223.0,222 HEADY DR,NASHVILLE,TN,...,H R Properties Of Tn Llc,CARN,CAA01R301,1728297,"For every 30 feet of street frontage, or fract...",34.0,37018500.0,496193.07,22479.342187,(5) >120%_AMI


In [171]:
rebuilds.drop(rebuilds.loc[rebuilds['ConstCost'] == 0].index, inplace=True)
rebuilds.drop(rebuilds.loc[rebuilds['ConstCost'] == 1].index, inplace=True)
#rebuilds.drop(rebuilds.loc[rebuilds['PermType'] == ['CATR', 'CADM', '']])

In [176]:
rebuilds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2276 entries, 44 to 99801
Data columns (total 22 columns):
PermitNo                   2276 non-null object
PermTypeDesc               2276 non-null object
PermSubDesc                2276 non-null object
Parcel                     2276 non-null object
EnteredDt                  2276 non-null datetime64[ns]
IssuedDt                   2276 non-null datetime64[ns]
ConstCost                  2276 non-null float64
Address                    2276 non-null object
City                       2263 non-null object
State                      2263 non-null object
ZIP                        2276 non-null int64
Subdiv_Lot                 2276 non-null object
Contact                    2276 non-null object
PermType                   2276 non-null object
PermSubtype                2276 non-null object
IVRTrk#                    2276 non-null int64
Purpose                    2276 non-null object
CouncilDist                2276 non-null float64
CensusTract

In [177]:
rebuilds_dt = rebuilds.set_index('IssuedDt')

In [175]:
rebuilds_dt.PermType.plot()

TypeError: Empty 'DataFrame': no numeric data to plot

In [144]:
rebuilds.bucket.plot(kind = 'bar')
plt.bar()

TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
rebuilds.Yearly_Housing_Spending.min()

In [ ]:
rebuilds.sort_values('Yearly_Housing_Spending')

In [ ]:
rebuilds.Yearly_Housing_Spending.plot(kind='hist', bins=150)

In [ ]:
rebuilds.Yearly_Housing_Spending.quantile(0.6)

In [ ]:
#rebuilds.set_index('IssuedDt', inplace=True)

In [ ]:
rebuilds.drop(rebuilds.loc[rebuilds.ConstCost > 3.678022e+05].index, inplace=True)

In [ ]:
rebuilds.info()

In [ ]:
homes.info()

In [ ]:
nonhomes = homes.loc[homes['ConstCost'] == 0.0].index

In [ ]:
homes.drop(index=nonhomes, inplace=True)

In [ ]:
permitdecode = pd.DataFrame(issued[['PermTypeDesc','PermType']])

In [ ]:
result = re.findall('-?\d*\.\d*', issued.MappedLoc[1])

In [ ]:
print(result)

In [ ]:
permitdecode.drop_duplicates()

CARN, CARR, CADM

In [ ]:
#resnew = pd.DataFrame(issued.loc[issued['PermType'] == 'CARW'])
respermits = pd.DataFrame(issued.loc[issued['PermType'].isin(['CARN','CARR'])])

In [ ]:
respermits.info()

In [ ]:
newhomes = respermits.Purpose.str.match(r'[dD]emolition')

In [ ]:
respermits.Purpose.unique()

In [ ]:
demos = pd.DataFrame(issued.loc[issued['PermType'] == 'CADM'])

In [ ]:
demos.info()

In [ ]:
issued.loc[issued['Address'] == '914 ACKLEN AVE']

In [ ]:
demos.head()

In [ ]:
demos.ConstCost.median()

In [ ]:
respermits.set_index('IssuedDt', inplace=True)

In [ ]:
respermits.info()

In [ ]:
len(respermits.PermSubDesc.unique())

In [ ]:
monthlycost = respermits['ConstCost'].resample('M').mean()

In [ ]:
monthlycost.plot()

In [ ]:
respermits.

In [ ]:
respermits.describe()

In [ ]:
resnew.EnteredDt.describe()

In [ ]:
resnew.IssuedDt.describe()

In [ ]:
resnew.ConstCost.describe()

In [ ]:
issued['CouncilDist'].unique()

In [ ]:
issued['MappedLoc'][1]

In [ ]:
issued['extcoords'] = issued.MappedLoc.str.extract(r'?\(.*\)', expand=False)

In [ ]:
coords = re.compile(r'\(.*\)')

In [ ]:
matches = []

In [ ]:
for string in issued.MappedLoc:
    match = re.findall(coords, string)
    matches.append(match)

In [ ]:
type(matches)

In [ ]:
type(matches[1])

In [ ]:
#[s for s in i[s] for i in matches]

coordlist = []
for i in enumerate(matches):
    i = []
    for s in i[s]:
        coordlist.append(tuple(s))

In [ ]:
len(matches)

In [ ]:
matches[1]

In [ ]:
issued['coords'] = pd.Series(matches).astype(tuple)

In [ ]:
issued['coords'][1]

Income data:
    Median average income: 68000
        Income subgroups:
        30% MAI (20400 /yr) (30% housing = 510/mo)
        60% MAI (40800 /yr) (30% housing = 1020/mo)
        80% MAI (54400 /yr) (30% housing = 1360/mo)
        120% MAI (81600 /yr) (30% housing = 2040/mo)

Model development:
    
    Owners = 54%
        - Model annual adjustments to these numbers
    
    
    Median Housing Cost - Owner = 1336/month * 12 = 16032/yr

    15-yr owner gap (affordable) = 3367
    15-yr owner gap (market) = 35751

    10-yr predicted owner gap (affordable) = 1257
    10-yr predicted owner gap (market) = 41421
    

In [ ]:
issued.Issued.min()

In [ ]:
issued.Issued.max()

In [ ]:
issued['monthlies'] = pd.Series([cost/360 for cost in issued.ConstCost])

In [ ]:
issued.info()

In [ ]:
issued.Zip.unique()

In [ ]:
r = requests.get('https://api.census.gov/data.html', auth=('davidwhainesiii@gmail.com', '83dffa3105cb08ef0e77c24ff28a1dc62272ae48'))

In [ ]:
type(r)

In [ ]:
rawresponse = r.text
censussoup = BS(rawresponse, 'html.parser')

In [ ]:
censussoup.find_all('table')

Question 1: At what point in time did the Codes Administration decide to begin “closing the loophole” by converting R zoning to RS?

Response 1: The Codes Administration does not set zoning policy, we interpret/enforce it. The Planning Commission and Metro Council are the ones who change zoning rules. The last major revision countywide to the zoning code was in 1997 when they rezoned a lot of properties in bulk. Since then, Councilmembers have proposed zoning changes for their neighborhoods on a piecemeal basis, or developers have requested zoning changes for particular properties. There hasn’t been an effort by Metro Government to downzone properties because of the duplex situation, but some Council members have. (edited)

Question 2: According to the report (Housing Nashville), there are predetermined amounts of needed housing and rental properties in each district. How is the split between housing demand and rental demand being evaluated? Is this arbitrary or is there census data giving us a proportion to go off of?

Question 3: What criteria was used to calculated the ‘Maximum Affordable Sales Price’  from the MHI on page 56 of the Housing Nashville Report? What interest rate and/or tax rate was assumed?

Response 2 & 3: Unfortunately, the folks that produced the Housing Nashville Report have left Metro Government so I don’t know that we’ll get all the answers. Looking at question three, I believe the estimate was based on a 5% down payment and about a 4% interest rate for a 30 year mortgage.
Also, We have also asked Assessments for an update to the 3-years of assessment data to include the square footage of the property and the zoning code for the property. (edited)

Renters = 46%
Median Housing Cost - Renter = 874/month * 12 = 10488/yr
    
    15-yr rental gap (affordable) = 17754
    15-yr rental gap (market) = 22219
        
    10-yr predicted rental gap (affordable) = 30934
    10-yr predicted rental gap (market) = 23043

In [ ]:
codetypes = pd.read_excel('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/Codes-Type-Subtype-List-20181011.xlsx')

In [ ]:
codetypes.head()

In [ ]:
codetypes.info()

In [ ]:
codetypes.CASE_TYPE_DESC.unique()

In [ ]:
codetypes.loc[codetypes.CASE_TYPE_DESC == 'Building Demolition Permit']

In [ ]:
pwd

In [ ]:
counties = gpd.read_file('./metrodata/cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')

In [ ]:
counties.plot()

In [ ]:
counties.head()